In [ ]:
import discord
import asyncio
import datetime
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud
from PIL import Image
import pandas as pd
import nltk
from konlpy.tag import Mecab
from discord.ext import commands
import re
import pymongo

token = ""
bot = commands.Bot(command_prefix='!')  #command_prefix : the first one to match is the invocation prefix


@bot.event
async def on_ready():
    print('Ready!!')


@bot.command()
async def news(ctx, p_date):
    # 이미지 파일 불러오기 위한 엠베드 추가
    embed = discord.Embed(
        title='wordcloud',
        color=discord.Color.blue()
    )
    datas = tdk(p_date)
    # 단어 총 갯수
    number = 0
    for data in datas:
        number += data[1]
    # %구하기
    words = []
    for data in datas[:10]:
        words.append(data[0] + " " + str(round(data[1] / number * 100, 2)) + "%")
    wordstr = " ".join(words)
    file = discord.File("1.png", filename="image.png")
    embed.set_image(url="attachment://image.png")
    # %정보 엠베드에 붙이기
    embed.add_field(name="word %", value=wordstr, inline=False)
    # 엠베드 보내기
    await ctx.send(file=file, embed=embed)


@bot.command()
async def titles(ctx, p_date):
    df = database(p_date)
    for data in df.title:
        await ctx.send(data)


# 데이터 베이스에서 추출하여 데이터프레임 만드는 함수
def database(p_date):
    client = pymongo.MongoClient('mongodb://')
    db = client.news
    ls = list(db.articles.find({'p_date': p_date}))
    df = pd.DataFrame(ls)
    return df


# 오늘의 키워드
def tdk(p_date):
    # 자연어처리 툴
    mecab = Mecab()
    # 데이터프레임 불러오기
    df = database(p_date)
    # 기사 데이터 추출
    contents = []
    for data in df.content:
        if type(data) == str:
            contents.append(data.strip())
    # 단어만 추출
    nouns = []
    for idx in range(len(contents)):
        nouns.extend(mecab.nouns(contents[idx]))
    # 불용어처
    with open('/Users/Downloads/ko_stopwords.txt', 'rt') as txt_file:
        stop_words = txt_file.readlines()
    stop_word = []
    for idx in range(len(stop_words)):
        stop_word.append(stop_words[idx].replace("\n", ""))

    new_nouns = [each_word for each_word in nouns if each_word not in stop_word]
    # 단어 중복 체크
    words = nltk.Text(new_nouns, name='words')
    # 상위100개 추출
    data = words.vocab().most_common(100)
    # 워드크라우드 만들기
    wordcloud = WordCloud(font_path='/Users/Documents/NanumBarunGothic.ttf',
                          relative_scaling=0.05,
                          background_color='white',
                          ).generate_from_frequencies(dict(data))
    wordcloud.to_file('1.png')
    return data


bot.run(token)